In [ ]:
# 구글 드라이브 접근 코드
from google.colab import drive
drive.mount('/content/drive')
My_path = '/content/drive/MyDrive/LG_Academy/AI_Code/'

## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

## 데이터 불러오기

In [ ]:
DataLength = 2774
NoOfSensor = 3
NoOfData   = 180

DataLength, NoOfSensor

In [ ]:
NormalSet   = np.zeros([NoOfData, DataLength*NoOfSensor])
AbnormalSet = np.zeros([NoOfData, DataLength*NoOfSensor])

for i in range(NoOfData):
    tempNormalPath   = My_path + './SpotWeldingData/Normal_%d'  %(i+1)
    tempAbnormalPath = My_path + './SpotWeldingData/Abnormal_%d'%(i+1)

    tempNormal       = np.array(pd.read_csv(tempNormalPath  , header = None)).reshape(-1)
    tempAbnormal     = np.array(pd.read_csv(tempAbnormalPath, header = None)).reshape(-1)

    NormalSet[i, :]        = tempNormal
    AbnormalSet[i, :]      = tempAbnormal


NormalSet.shape, AbnormalSet.shape

In [ ]:
# 학습: 정상 데이터만 사용

TrainData = NormalSet[:-30, :]
ValidData = np.concatenate([NormalSet[-30:, :], AbnormalSet[-30:, :]], axis=0)

TrainData.shape, ValidData.shape

## Isolation Forest 모델 구성

In [ ]:
IsoForest = IsolationForest(n_estimators=100,   # 트리의 갯수
                            max_samples='auto', # 학습 데이터셋 중 학습에 사용할 데이터 갯수
                            contamination=0.01, # 학습 데이터 중 Potential anomaly로 설정할 비율
                            random_state=777    # 재현성 확보
                            )

## Isolation Forest 모델 학습 및 평가

In [ ]:
# 모델 학습
IsoForest.fit(TrainData)

In [ ]:
# 모델 활용 (학습데이터)
Prediction_Train = IsoForest.predict(TrainData)
Prediction_Train

## 검증 데이터 이상탐지

In [ ]:
# 모델 활용 (검증데이터) 이상탐지 결과 (1 정상, -1 이상)
Prediction_Valid = IsoForest.predict(ValidData)
Prediction_Valid

In [ ]:
# 모델 활용 (검증데이터) Anomaly score 확인
AnoScores_Valid = IsoForest.decision_function(ValidData)
AnoScores_Valid

In [ ]:
# 검증 데이터 Anomaly score 분할
NoOfValidData   = int(ValidData.shape[0]/2)

Valid_Normal   = AnoScores_Valid[:NoOfValidData]
Valid_Abnormal = AnoScores_Valid[NoOfValidData:]

In [ ]:
# PDF 그래프
bins = np.arange(-0.12, 0.12, 0.01)

plt.figure(figsize=(10,6))
plt.hist(Valid_Normal, label = 'Valid Normal', edgecolor='k', bins = bins, alpha = 0.7, density = True, color = 'C2')
plt.hist(Valid_Abnormal, label = 'Valid Abnormal', edgecolor='k', bins = bins, alpha = 0.7, density = True, color = 'C3')

plt.grid(alpha = 0.8)
plt.xlabel('Reconstruction Error')
plt.ylabel('Density')
plt.legend(loc='upper right')
plt.title('PDF of Reconstruction')
plt.show()